<a href="https://colab.research.google.com/github/achiral/YouTubeCapE2J/blob/main/YouTubeCapE2J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 音声・動画ファイルから文字起こし、翻訳、字幕挿入
@sakasegawa（https://qiita.com/sakasegawa) 様、Nego_t (https://zenn.dev/nego_t/articles/7c2c11f9bed467) 様の記事を参考に、少し編集を加えました。  
**処理の内容自体は、まったく同じ**です。  
以下の操作をできるように整理したものです。
- Google Driveに保存してある動画ファイル、音声ファイルを利用
- 音声ファイルの処理
  - 文字起こし
  - 翻訳（英語から日本語）
  - テキストファイルへの書き出し
- 動画ファイルの処理
  - 対応する音声ファイルが必要
    - FFmpegで音声ファイルを分離
    - 自作動画であればYouTubeにアップロードして一括処理可能
  - 字幕挿入
    - 動画に対応した音声ファイルが必要
    - 動画からFFmpegにより音声分離

## オリジナルサイト
- 概要  
YouTube英語動画に自動で字幕をつける(動画ライセンスに注意)
https://qiita.com/sakasegawa/items/1eae5038cfbd09baad44

- 使い方  
https://qiita.com/sakasegawa/items/1eae5038cfbd09baad44

- サンプル  
https://www.youtube.com/watch?v=R-qLOUTHP5Q

In [1]:
#@title パスの設定
# YouTubeのvidを入力する(入力しない場合はデフォルト)
url = "https://www.youtube.com/watch?v=2uIhgCcGr4M"#@param {type:"string"}{allow-input: true} ["YouTubeのURLを入力してください","https://www.youtube.com/watch?v=2uIhgCcGr4M","youtube.com/watch?v=bK6g_TbW7XM","https://www.youtube.com/watch?v=WRxN3omvrf0", "https://www.youtube.com/watch?v=[INPUT]"]
# 動画はPlayground/videos以下に保存される
video_folder = "/content/drive/MyDrive/movietranslate/videos/"#@param {type:"string"}{allow-input: true}["インプットファイルのフォルダパスを入力してください", "/content/drive/MyDrive/movietranslate/videos/"]
# フォントは以下のフォントをPlayground/fonts以下に置く
# https://flopdesign.booth.pm/items/2328262
# LightNovelPOPv2.otf として保存
font_folder = "/content/drive/MyDrive/movietranslate/fonts/"#@param {type:"string"}{allow-input: true}["アウトプット先のフォルダパスを入力してください", "/content/drive/MyDrive/movietranslate/fonts/"]
if url == "":
  url = "https://www.youtube.com/watch?v=WRxN3omvrf0"

print("video_folder = " + video_folder)
print("font_folder = " + font_folder)
print("url = " + url)

video_folder = /content/drive/MyDrive/movietranslate/videos/
font_folder = /content/drive/MyDrive/movietranslate/fonts/
url = https://www.youtube.com/watch?v=2uIhgCcGr4M


In [2]:
#@title url to vid
import urllib.parse
qs = urllib.parse.urlparse(url).query
qs_d = urllib.parse.parse_qs(qs)
vid = qs_d['v'][0] #@param ["qs_d['v'][0]", "拡張子を除いたファイル名を入力してください(例)WRxN3omvrf0", "'WRxN3omvrf0'"] {type:"raw", allow-input: true}
print("url: https://youtube.com/watch?v=%s" % vid)
print("vid: " + vid)

url: https://youtube.com/watch?v=2uIhgCcGr4M
vid: 2uIhgCcGr4M


In [3]:
#@title nvidia確認
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-0dac2195-6ad7-c896-18db-d741f1d946ef)


In [4]:
#@title Mount Google Drive
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

import os
if not os.path.exists("/content/drive/MyDrive/movietranslate/fonts/LightNovelPOPv2.otf"):
    raise Exception("フォルダを作成してフォントをダウンロードしてください")
print("○フォントフォルダーの内容：")
%ls /content/drive/MyDrive/movietranslate/fonts

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
○フォントフォルダーの内容：
LanobePOPv2.zip  LightNovelPOPv2.otf


In [5]:
#@title インストール
# youtube: 動画ダウンロード
!pip install yt_dlp
# 音声処理系: whisper
!pip install pydub
!pip install git+https://github.com/openai/whisper.git
# 動画処理系: moviepy
!pip install -q moviepy
!apt install imagemagick
!pip install imageio==2.4.1
# その他: いい感じの改行
!pip install budoux
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 18.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-7hfda9_f
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-7hfda9_f
  Resolved https://github.com/openai/whisper.git to commit 7858aa9c08d98f75575035ecd6481f462d66ca27
  

In [6]:
#@title おまじない
# https://github.com/Zulko/moviepy/issues/693#issuecomment-622997875
!sed -i '/<policy domain="path" rights="none" pattern="@\*"/d' /etc/ImageMagick-6/policy.xml

In [7]:
#@title 音声処理: ダウンロード (ダウンロード済みファイルがあれば処理不要)
import yt_dlp
from pydub import AudioSegment

# 動画全体の音声をダウンロード
# videos/{vid}.m4a として出力している
def music_download(vid):
    output_file_name = video_folder + f"{vid}.m4a"
    video_url = f"https://www.youtube.com/watch?v={vid}"

    ydl_opts = {
        "format": "m4a/bestaudio/best",
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "m4a",
            }
        ],
        "outtmpl": output_file_name,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download(video_url)
    
    audio = AudioSegment.from_file(output_file_name, "m4a")
    audio_file_name = video_folder + f"{vid}.wav"
    audio.export(audio_file_name, format="wav")

In [8]:
#@title 動画処理: ダウンロード (ダウンロード済みファイルがあれば処理不要)

import yt_dlp
from yt_dlp.utils import download_range_func

# videos/{vid}.mp4 として出力している
def video_download(vid):
    output_file_name = video_folder + f"{vid}.mp4"
    video_url = "https://www.youtube.com/watch?v=%s" % vid

    ydl_opts = {
        "format": "bestvideo[ext=mp4]+bestaudio[ext=mp4]/mp4+best[height<=480]",
        "outtmpl": output_file_name
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download(video_url)

In [9]:
#@title whisper
# 音声処理: whisper
import whisper

def generate_transcribe_wav(vid):
    # Whisper高速化テクニック
    # https://qiita.com/halhorn/items/d2672eee452ba5eb6241
    model = whisper.load_model("large", device="cpu")
    _ = model.half()
    _ = model.cuda()

    # exception without following code
    # reason : model.py -> line 31 -> super().forward(x.float()).type(x.dtype)
    for m in model.modules():
        if isinstance(m, whisper.model.LayerNorm):
            m.float()
    target_file_name = video_folder + f"{vid}.wav"
    result = model.transcribe(target_file_name)
    return result

def generate_transcribe_mp3(vid):
    # Whisper高速化テクニック
    # https://qiita.com/halhorn/items/d2672eee452ba5eb6241
    model = whisper.load_model("large", device="cpu")
    _ = model.half()
    _ = model.cuda()

    # exception without following code
    # reason : model.py -> line 31 -> super().forward(x.float()).type(x.dtype)
    for m in model.modules():
        if isinstance(m, whisper.model.LayerNorm):
            m.float()
    target_file_name = video_folder + f"{vid}.mp3"
    result = model.transcribe(target_file_name)
    return result

In [10]:
#@title 翻訳モデル: fugumt
from transformers import pipeline

translator = pipeline('translation', model='staka/fugumt-en-ja')

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
#@title 動画処理: 字幕(日本語の場合は seg_text: 「seg_text」 を選択) {display-mode: "form"}
from moviepy import editor
from moviepy.video.io.VideoFileClip import VideoFileClip
import budoux

def generate_subtitle(vid, transcribe, max_line_length = 14, max_lines = 2):
    input_file_name = video_folder + f'{vid}.mp4'             #@param ["video_folder + f'{vid}.mp4'", "パスを入力するか選択してください: /content/drive/MyDrive/movietranslate/videos/FILENAME.mp4", "'/content/drive/MyDrive/movietranslate/videos/WRxN3omvrf0.mp4'"] {type:"raw", allow-input: true}
    output_file_name = video_folder + f'{vid}_subtitle.mp4' #@param ["video_folder + f'{vid}_subtitle.mp4'", "パスを入力するか選択してください: /content/drive/MyDrive/movietranslate/videos/FILENAME_subtitle.mp4", "'/content/drive/MyDrive/movietranslate/videos/WRxN3omvrf0_subtitle2.mp4'"] {type:"raw", allow-input: true}
    video = VideoFileClip(input_file_name)
    annotated_clips = []
    last_end = 0
    for segment in transcribe['segments']:
        seg_start_sec = int(segment['start'])
        seg_end_sec = int(segment['end'])
        seg_text = segment['text']
        # 翻訳
        result = translator(seg_text)
        seg_text = result[0]['translation_text'] #@param ["result[0]['translation_text']", "seg_text"] {type:"raw", allow-input: true}
        # 字幕がない場所はそのまま動画を入れる
        if last_end != seg_start_sec:
            print("skip: %s to %s" % (last_end, seg_start_sec))
            annotate_seg = video.subclip(last_end, seg_start_sec)
            annotated_clips.append(annotate_seg)

        # 文章が長すぎる場合は分割しつつ字幕として表示する
        seg_all_sec = seg_end_sec - seg_start_sec
        parsed_text_list = sentence_parse_and_line_parse(seg_text, max_line_length, max_lines)
        section_per_sec = int(seg_all_sec / len(parsed_text_list))
        print("jimaku: %s to %s" % (seg_start_sec, seg_end_sec))
        if section_per_sec == 0:
            section_per_sec = 1
        for i, parsed_text in enumerate(parsed_text_list):
            section_start_sec = seg_start_sec + section_per_sec * i
            section_end_sec = seg_start_sec + section_per_sec * (i + 1)
            # 一番ケツはend_secに合わせる
            if i == (len(parsed_text_list) - 1):
                section_end_sec = seg_end_sec
                print("    ketsu: %s" % section_end_sec)
            # 最後ちょびっとでも超えるとエラーなので一番最後はNoneにする
            if section_end_sec >= video.duration:
                section_end_sec = None
            annotate_seg = annotate(video.subclip(section_start_sec, section_end_sec), parsed_text)
            print("    section start: %s, end: %s, text: %s" % (section_start_sec, section_end_sec, parsed_text))
            annotated_clips.append(annotate_seg)

        # last_end: 前回の終わり秒の参照用
        last_end = seg_end_sec
        print(last_end)
    
    print(f"last_end is {last_end} and video.duration is {video.duration}")
    if last_end != None and video.duration >= last_end:
        print("skip: %s to %s" % (last_end, video.duration))
        annotate_seg = video.subclip(last_end, None)
        annotated_clips.append(annotate_seg)
    final_clip = editor.concatenate_videoclips(annotated_clips)
    final_clip.write_videofile(output_file_name, audio_codec="aac")

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2367488/45929032 bytes (5.2%)5963776/45929032 bytes (13.0%)9150464/45929032 bytes (19.9%)12869632/45929032 bytes (28.0%)16056320/45929032 bytes (35.0%)19775488/45929032 bytes (43.1%)23060480/45929032 bytes (50.2%)25837568/45929032 bytes (56.3%)29589504/45929032 bytes (64.4%)32636928/45929032 bytes (71.1%)35905536/45929032 bytes (78.2%)38526976/45929032 bytes (83.9%)

In [12]:
#@title 字幕の設定関数 {display-mode: "form" }
def annotate(clip, txt):
    # https://moviepy-tburrows13.readthedocs.io/en/improve-docs/ref/VideoClip/TextClip.html
    # 1外側
    txtclip1 = editor.TextClip(txt
        ,fontsize=75                                #@param {type:"integer", allow-input: true}
        ,font=font_folder + 'LightNovelPOPv2.otf'   #@param ["font_folder + 'LightNovelPOPv2.otf'", ""]{type:"raw", allow-input: true}
        ,color='white'                            #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
        ,stroke_color="white"                  #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
        ,stroke_width=12                           #@param {type:"integer", allow-input: true}
    )
    # 2内側
    txtclip2 = editor.TextClip(txt
        , fontsize=75                               #@param {type:"integer", allow-input: true}
        , font=font_folder + 'LightNovelPOPv2.otf'  #@param ["font_folder + 'LightNovelPOPv2.otf'", ""]{type:"raw", allow-input: true}
        , color='yellow'                              #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
        , stroke_color="red"                      #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
        , stroke_width=5                          #@param {type:"integer", allow-input: true}
    )
    # 3
    # txtclip3 = editor.TextClip(txt
    #     ,  fontsize=75                               #@param {type:"integer", allow-input: true}
    #     ,  font=font_folder + 'LightNovelPOPv2.otf'  #@param ["font_folder + 'LightNovelPOPv2.otf'", ""]{type:"raw", allow-input: true}
    #     ,  color='white'                              #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
    #     ,  stroke_color="white"                      #@param ["#1c55be", 'white', "light gray", "red", "black", "yellow", 'blue', 'green']{type:"string", allow-input: true}
    #     ,  stroke_width=12                          #@param {type:"integer", allow-input: true}
    # )
    cvc = editor.CompositeVideoClip([clip
                                     ,txtclip1.set_pos(('center', 'bottom'))
                                     ,txtclip2.set_pos(('center', 'bottom'))
                                    #  ,txtclip3.set_pos(('center', 'bottom'))
                                     ])
    return cvc.set_duration(clip.duration)

# 文章をいい感じに分割・改行する
# 例
# input: "いいね あのあくまでそのすでにその下預金で2600万円とか置いてる人であれば多分500万円ぐらい"
# output: ['いいね', 'あのあくまでそのすでにその\n下預金で2600万円とか', '置いてる人であれば\n多分500万円ぐらい']
def sentence_parse_and_line_parse(text, max_line_length = 14, max_lines = 2):
    # スペースで分割
    space_parsed_text_list = text.split(' ')
    parsed_text_list = []
    for space_parsed_text in space_parsed_text_list:
        parsed_text_list += text_line_parse(space_parsed_text, max_line_length, max_lines)
    return parsed_text_list

# 任意の行数ごとにわける
# 例
# input: "あのあくまでそのすでにその下預金で2600万円とか置いてる人であれば多分500万円ぐらい"
# output: ['あのあくまでそのすでにその\n下預金で2600万円とか', '置いてる人であれば\n多分500万円ぐらい']
def text_line_parse(text, max_length = 14, max_lines = 2):
    text_list = budoux_parse_text(text, max_length, [])
    line_parsed_text_list = []
    for i in range(0, len(text_list), max_lines):
        text_with_2lines = text_list[i: i+max_lines]
        text = "\n".join(text_with_2lines)
        line_parsed_text_list.append(text)
    return line_parsed_text_list

# budouxで任意の文字数ごとにわける
# 最後の引数は再帰用
# 例
# input: "あのあくまでそのすでにその下預金で2600万円とか置いてる人であれば多分500万円ぐらい"
# output: ['あのあくまでそのすでにその', '下預金で2600万円とか', '置いてる人であれば', '多分500万円ぐらい']
def budoux_parse_text(text, max_length = 14, text_list = []):
    budoux_parser = budoux.load_default_japanese_parser()
    budoux_parsed_text_list = budoux_parser.parse(text)
    parsed_text = ""
    overflow_text = ""
    for budoux_parsed_text in budoux_parsed_text_list:
        if max_length > len(parsed_text + budoux_parsed_text):
            parsed_text += budoux_parsed_text
        elif len(parsed_text) == 0 and len(budoux_parsed_text) > max_length:
            parsed_text = budoux_parsed_text[0:max_length]
            overflow_text += budoux_parsed_text[max_length:]
        else:
            overflow_text += budoux_parsed_text
    text_list.append(parsed_text)
    if len(overflow_text) > 0:
        print(overflow_text)
        budoux_parse_text(overflow_text, max_length, text_list)
    return text_list

In [13]:
#@title 動画から音声（mp3）抽出 (YouTubeから処理する場合は不要)
input_path = video_folder + vid + '.mp4' #@param ["video_folder + vid + '.mp4'"] {type:"raw", allow-input: true}
out_path = video_folder + vid + '.mp3' #@param ["video_folder + vid + '.mp3'", "video_folder + vid + '.wav'", "video_folder + vid + '.aac'"] {type:"raw", allow-input: true}

# !ffmpeg -i $input_path -vn -acodec copy $out_path
# !ffmpeg -i $input_path -vn -acodec copy $out_path
!ffmpeg -i $input_path -acodec libmp3lame -ab 256k $out_path

print("「" + input_path + "」の動画ファイルから")
print("「" + out_path + "」に音声抽出しました")

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [14]:
#@title 音声・動画ダウンロード (ダウンロード済みファイルがあれば処理不要: コメントアウトする), 音声書き出し
# URLの確認
print("◯対象URL")
print("https://youtube.com/watch?v=%s" % vid)
# 音声をダウンロードする
print("◯音声ダウンロード")
print("...")
music_download(vid)
print("音声ダウンロード完了!")
# 音声書き出し
print("◯音声書き出し")
print("...")
transcribe = generate_transcribe_wav(vid)       #@param ["generate_transcribe_wav(vid)", "generate_transcribe_mp3(vid)"] {type:"raw", allow-input: true}
print("音声書き出し完了！")
# 動画をダウンロードする
print("◯動画ダウンロード")
video_download(vid)

◯対象URL
https://youtube.com/watch?v=2uIhgCcGr4M
◯音声ダウンロード
...
[youtube] Extracting URL: https://www.youtube.com/watch?v=2uIhgCcGr4M
[youtube] 2uIhgCcGr4M: Downloading webpage
[youtube] 2uIhgCcGr4M: Downloading android player API JSON
[info] 2uIhgCcGr4M: Downloading 1 format(s): 140
[download] /content/drive/MyDrive/movietranslate/videos/2uIhgCcGr4M.m4a has already been downloaded
[download] 100% of  924.27KiB
[ExtractAudio] Not converting audio /content/drive/MyDrive/movietranslate/videos/2uIhgCcGr4M.m4a; file is already in target format m4a
音声ダウンロード完了!
◯音声書き出し
...


100%|██████████████████████████████████████| 2.87G/2.87G [00:27<00:00, 112MiB/s]


音声書き出し完了！
◯動画ダウンロード
[youtube] Extracting URL: https://www.youtube.com/watch?v=2uIhgCcGr4M
[youtube] 2uIhgCcGr4M: Downloading webpage
[youtube] 2uIhgCcGr4M: Downloading android player API JSON
[info] 2uIhgCcGr4M: Downloading 1 format(s): 22
[download] Destination: /content/drive/MyDrive/movietranslate/videos/2uIhgCcGr4M.mp4
[download] 100% of    7.07MiB in 00:00:12 at 586.47KiB/s 


In [15]:
#@title save original text
print("○google driveに書き出します○")
output_file = vid + ".txt"                #@param ['vid + ".txt"', ''] {type:"raw", allow-input: true}
txt_path = video_folder + output_file
with open(txt_path, mode='w') as f:
  f.write(transcribe["text"])

print("●「" + txt_path + "」 に以下のテキストを出力しました●")
print(transcribe["text"])

○google driveに書き出します○
●「/content/drive/MyDrive/movietranslate/videos/2uIhgCcGr4M.txt」 に以下のテキストを出力しました●
 So, AIR generation is awesome, but your images are too small. Not a problem. Anytime I really like an image, I use another free AI to redraw it bigger. There's a lot of AI-based image-size websites out there, but the one I usually use is called Upscale Media. Simply upload your image, give it a few seconds, and you'll have the option to make it bigger. You'll also have the option to have the AI redraw and increase the quality. And since it's free, I usually just do both. Download the image, you're done. Now there's a lot more people using the website now, so it takes a little bit longer. But if you sign up and make a free account, you will be able to upscale 45 images every month, which is totally worth the no money it takes. Just upload, go to storage, click the image, transform, transformations, upscale, hit refresh, and download your image. So, hope that helps, and as always, hope

In [16]:
#@title translation (E2J) -> save translated text
# translation (E2J)
print("○翻訳中です(English to Japanese)")
result_translated = translator(transcribe['text'], max_length=650)
print("●翻訳が終了しました●")
# save
print("○google driveに書き出します○")
output_file = vid + "_translated.txt"   #@param ['vid + "_translated.txt"', ''] {type:"raw", allow-input: true}
txt_path = video_folder + output_file
with open(txt_path, mode='w') as f:
  f.write(result_translated[0]["translation_text"])

print("●「" + txt_path + "」 に以下のテキストを出力しました●")
print(result_translated[0]["translation_text"])

○翻訳中です(English to Japanese)
●翻訳が終了しました●
○google driveに書き出します○
●「/content/drive/MyDrive/movietranslate/videos/2uIhgCcGr4M_translated.txt」 に以下のテキストを出力しました●
AIR生成は素晴らしいものですが あなたの画像は小さすぎます 問題ありません 画像が好きなときはいつでも 別の無料のAIを使って より大きく描き直します AIベースの画像サイズのウェブサイトはたくさんありますが 普段はUpscale Mediaと呼ばれています 画像をアップロードして 数秒で拡大できます AIの再描画と画質向上のオプションもあります 両方が無料です イメージをダウンロードすればいいのです 今はもっと多くの人がWebサイトを使っているので もう少し時間がかかります しかし 登録して無料でアカウントを作成すれば 毎月45枚の画像をスケールアップできます お金がかからない価値があります アップロードし 画像をクリックし 変換し 変換し アップスケールし アップスケールし ダウンロードし 期待を更新します 素晴らしい日が来るのです


In [17]:
#@title 動画に字幕データをつけて出力する
print("◯字幕生成")
generate_subtitle(vid, transcribe)

◯字幕生成
素晴らしいですが、あなたの小さすぎます。
小さすぎます。
jimaku: 0 to 4
    section start: 0, end: 2, text: だから、AIR生成は画像は
素晴らしいですが、あなたの
    ketsu: 4
    section start: 2, end: 4, text: 小さすぎます。
4
jimaku: 4 to 5
    ketsu: 5
    section start: 4, end: 5, text: 問題ありません。
5
いつでも、別の無料のAIを使ってより大きく描き直します。
使ってより大きく描き直します。
描き直します。
jimaku: 5 to 9
    section start: 5, end: 7, text: 画像が本当に好きなときは
いつでも、別の無料のAIを
    ketsu: 9
    section start: 7, end: 9, text: 使ってより大きく
描き直します。
9
ウェブサイトはたくさんありますが
ありますが
jimaku: 9 to 13
    section start: 9, end: 11, text: AIを使った画像サイズの
ウェブサイトはたくさん
    ketsu: 13
    section start: 11, end: 13, text: ありますが
13
jimaku: 13 to 14
    ketsu: 14
    section start: 13, end: 14, text: 高級メディア。
14
数秒で拡大できます。
jimaku: 14 to 18
    ketsu: 18
    section start: 14, end: 18, text: 画像をアップロードして
数秒で拡大できます。
18
品質を向上させるオプションもあります。
オプションも
jimaku: 18 to 22
    section start: 18, end: 20, text: また、AIを再描画して
品質を向上させるあります。
    ketsu: 22
    section start: 20, end: 22, text: オプションも
22
無料ですので、両方のことをします。
します。
jimaku: 22 t

100%|██████████| 1289/1289 [00:05<00:00, 222.36it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/drive/MyDrive/movietranslate/videos/2uIhgCcGr4M_subtitle.mp4



 99%|█████████▉| 698/702 [00:41<00:00, 39.16it/s]WARNING:py.warnings:/usr/local/lib/python3.8/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/movietranslate/videos/2uIhgCcGr4M.mp4, 2764800 bytes wanted but 0 bytes read,at frame 701/702, at time 58.42/58.42 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

100%|██████████| 702/702 [00:41<00:00, 16.82it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/movietranslate/videos/2uIhgCcGr4M_subtitle.mp4 

